In [1]:
import typing as T  # pylint: disable=unused-import

import numpy as np
import numpy.typing as npt

from pydrake.solvers import (  # pylint: disable=import-error, no-name-in-module, unused-import
    MakeSemidefiniteRelaxation,
    MathematicalProgram,
    MathematicalProgramResult,
    Solve,
    IpoptSolver,
    SnoptSolver,
    GurobiSolver,
    MosekSolver,
    MosekSolverDetails,
    SolverOptions,
    CommonSolverOption,
)

from pydrake.geometry.optimization import (  # pylint: disable=import-error, no-name-in-module
    HPolyhedron,
    Point,
    ConvexSet,
    GraphOfConvexSets,
    GraphOfConvexSetsOptions,
)

from pydrake.symbolic import ( # pylint: disable=import-error, no-name-in-module, unused-import
    Polynomial,
    Variable,
    Variables,
    Expression,
)  

from contact_cart import ContactCart

import plotly.graph_objects as go  # pylint: disable=import-error
from plotly.express.colors import sample_colorscale  # pylint: disable=import-error
import plotly.graph_objs as go
from plotly.subplots import make_subplots

INFO:drake:Solved GCS shortest path using Mosek with convex_relaxation=true and preprocessing=false and no rounding.
INFO:drake:Solved GCS shortest path using Gurobi with convex_relaxation=false and preprocessing=false and no rounding.


IPOPT took 0.068s
solve successful!
30.05553319750908
SolutionResult.kSolutionFound
Solver is IPOPT
-------------------
nonconvex QCQP relaxaion with MOSEK took 0.173s
solve successful!
6.391753123912942
SolutionResult.kSolutionFound
Solver is Mosek
time 0.16512799263000488
rescode 0
solution_status 1
-------------------
GCS CR took 0.012s
solve successful!
5.8674624822741865
SolutionResult.kSolutionFound
Solver is Mosek
time 0.005387067794799805
rescode 0
solution_status 1
-------------------
Set parameter Username
Academic license - for non-commercial use only - expires 2024-12-01
GCS MICP took 0.073s
solve successful!
6.391736113661061
SolutionResult.kSolutionFound
Solver is Gurobi
-------------------
convex took 0.006s
solve successful!
6.496548990758626
SolutionResult.kSolutionFound
Solver is Mosek
time 0.0010449886322021484
rescode 0
solution_status 1
5.4205934923218813
-------------------


In [4]:
cart = ContactCart(v0=-5)

x_trajs = []
v_trajs = []
f_trajs = []
u_trajs = []
lcp_trajs = []

colors = {"micp": "red", "sdp":"blue", "qcqp":"green", "ipopt" : "yellow"}

functions = [(cart.solve_with_ipopt(), "ipopt"), (cart.solve_nonconvex_qcqp_sdp_relaxation(),"sdp"), (cart.solve_micp_with_gcs(), "micp"), (cart.solve_qcqp_convex_diff_decomposition(), "qcqp")]

for (stuff, name) in functions:
    cost, x_traj, v_traj, f_traj, u_traj, lcp_v_traj = stuff
    x_trajs.append((x_traj, name))
    v_trajs.append((v_traj, name))
    f_trajs.append((f_traj, name))
    u_trajs.append((u_traj, name))
    lcp_trajs.append((lcp_v_traj, name))



INFO:drake:Solved GCS shortest path using Gurobi with convex_relaxation=false and preprocessing=false and no rounding.


IPOPT took 0.100s
solve successful!
38.73397270574933
SolutionResult.kSolutionFound
Solver is IPOPT
-------------------
nonconvex QCQP relaxaion with MOSEK took 0.180s
solve successful!
21.569730356418976
SolutionResult.kSolutionFound
Solver is Mosek
time 0.17226886749267578
rescode 0
solution_status 1
-------------------
GCS MICP took 0.120s
solve successful!
14.606542488444163
SolutionResult.kSolutionFound
Solver is Gurobi
-------------------
convex took 0.006s
solve successful!
18.301406194289292
SolutionResult.kSolutionFound
Solver is Mosek
time 0.0011599063873291016
rescode 0
solution_status 1
14.661451349765702
-------------------


In [5]:
fig = go.Figure()

time = np.array(list(range(cart.N)))

# Create subplots
fig = make_subplots(rows=5, cols=1)


for (x_traj, name) in x_trajs:
    fig.add_trace(go.Scatter(x=time, y=x_traj, mode='lines', name=name, line=dict(color=colors[name]) ), row=1, col=1)

for (x_traj, name) in v_trajs:
    fig.add_trace(go.Scatter(x=time, y=x_traj, mode='lines', name=name, line=dict(color=colors[name])), row=2, col=1)    

for (x_traj, name) in f_trajs:
    fig.add_trace(go.Scatter(x=time, y=x_traj, mode='lines', name=name, line=dict(color=colors[name])), row=3, col=1)    

for (x_traj, name) in u_trajs:
    fig.add_trace(go.Scatter(x=time, y=x_traj, mode='lines', name=name, line=dict(color=colors[name])), row=4, col=1)    

for (x_traj, name) in lcp_trajs:
    fig.add_trace(go.Scatter(x=time, y=x_traj, mode='lines', name=name, line=dict(color=colors[name])), row=5, col=1)    
# Update layout
fig.update_layout(height=1200, width=800, title_text="Solver Comparisons")

fig.update_yaxes(title_text="Position", row=1, col=1)
fig.update_yaxes(title_text="Velocity", row=2, col=1)
fig.update_yaxes(title_text="Normal Force", row=3, col=1)
fig.update_yaxes(title_text="Control", row=4, col=1)
fig.update_yaxes(title_text="LCP violation", row=5, col=1)

# Show plot
fig.show()

